In [22]:
'''
Gets text content for tweet IDs
'''

# standard
import simplejson as json
from __future__ import print_function
from Sending_email import Sending_email_Cls
import getopt
import logging
import os
import sys
import csv
import pandas as pd
from time import sleep


# import traceback
# third-party: `pip install tweepy`
import tweepy


last_percent_reported = None
successTweets = 0
failedTweets = 0
tweet_count = 0


def report(tweet_count, successTweets, failedTweets):
  print('Download Complete!')
  print("Report:")
  print("Total:%s, Success:%s, Failed:%s" % (tweet_count, successTweets, failedTweets))


def download_progress_hook(count, totalSize, successTweets, failedTweets):
  """A hook to report the progress of a download. This is mostly intended for users with
  slow internet connections. Reports every 5% change in download progress.
  """

  global last_percent_reported
  percent = int(count * 100 / totalSize + 1)

  if last_percent_reported != percent:
    if percent % 5 == 0:
      sys.stdout.write("%s%%" % percent)
      sys.stdout.flush()
    else:
      sys.stdout.write(".")
      sys.stdout.flush()
    last_percent_reported = percent
    if percent == 100:
      report(count, successTweets, failedTweets)




# global logger level is configured in main()
Logger = None

# For the following, Generate your own at https://apps.twitter.com/app
CONSUMER_KEY = "rHhRh1sHgdOk3Z7bae3U5OQvM"
CONSUMER_SECRET = "7YeBw1lkUTLrcbnnZ5mHCEopE1HqKKzLlY405fbb5kbUyLUUsZ"
OAUTH_TOKEN = "346104128-81ip2674UJNKQX9npcuwuDfhveGIAt9ZjrGreJVW"
OAUTH_TOKEN_SECRET = "UPPnR5XN88x9ad02dihX5vOCbd8VeL3D5464ir785May3"

def get_tweet_id(line):
    '''
    Extracts and returns tweet ID from a line in the input.
    '''
  
    text= line.split(',') #check which separtor is used in the dataset
#     print(text)
    return (text)

def get_tweets_single(twapi, idfilepath):
    '''
    Fetches content for tweet IDs in a file one at a time,
    which means a ton of HTTPS requests, so NOT recommended.

    `twapi`: Initialized, authorized API object from Tweepy
    `idfilepath`: Path to file containing IDs
    '''
    # process IDs from the file
    failedTweetsFileCsv = "tweet_failed_binaryCalsses.csv"
    global successTweets, failedTweets, tweet_count
    with open(failedTweetsFileCsv, 'wt') as failedTweetsHandler: # Dataset's name in which downloaded tweet-text will be saved
      tweetWriterFailed = csv.writer(failedTweetsHandler)
      tweetWriterFailed.writerow(['tweetID', 'Error-Message','tweet-class'])
      with open(idfilepath, 'r') as idfile:
          tweets_downloaded = []
          next(idfile)
          totalSize = len(pd.read_csv(idfilepath))
          # print(totalSize)
          for line in idfile:
              tweet_id, tweet_class = get_tweet_id(line)
#               print('Fetching tweet for ID %s', tweet_id)
              try:
#                   print(tweet_id)
#                   tweet_id = '739067333847572480'
                  tweet = twapi.get_status(tweet_id, include_entities=True, trim_user=False)
#                   print('%s,%s,%s' % (tweet_id, tweet.text.encode('UTF-8'),tweet_class))
#                   tweetWriter.writerow([tweet_id, tweet.text.encode('UTF-8'),(tweet_class).strip()])
#                   print(tweet)
                  temp_tweet = dict(tweet._json)
                  label = (tweet_class).strip()
                  ugc_item = {"ugc": {"label": label}}
                  temp_tweet.update(ugc_item)
                  tweets_downloaded.append(temp_tweet)
                  successTweets = successTweets + 1
                  tweet_count = tweet_count + 1
                  download_progress_hook(tweet_count, totalSize
                                         ,successTweets, failedTweets)
#                   sleep(1)
              except tweepy.TweepError as te:
#                   print('Twitter error: ', te.response.text)
#                     print("error occured")
#                   Logger.warn('Failed to get tweet ID %s: %s', tweet_id, te.message)
                  try:
                    tweetWriterFailed.writerow([tweet_id, te.response.text, (tweet_class).strip()])
                  except:
                    tweetWriterFailed.writerow([tweet_id, (tweet_class).strip()]) 
                  tweet_count = tweet_count + 1
                  failedTweets = failedTweets + 1
                  download_progress_hook(tweet_count, totalSize
                                         , successTweets, failedTweets)
                  pass
    successTweetsFileJson = 'binary_dataset.json'
    with open(successTweetsFileJson, 'w') as successTweetsFileJsonHandler:
      json.dump(tweets_downloaded, successTweetsFileJsonHandler)


def usage():
    print('Provide the file containing tweetIDs and labels of tweets')
    sys.exit()

def main():
    idfile = 'binary_classes_ids.csv' #provide complete path of the file
    if not os.path.isfile(idfile):
        print('Not found or not a file: %s' % idfile, file=sys.stderr)
        usage()

    # connect to twitter
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
    api = tweepy.API(auth)
    
    # hydrate tweet IDs
    get_tweets_single(api, idfile)
    send = Sending_email_Cls()
    subject_email = "Notification: Downloading tweets\' Task."
    body_email = "Running task of downloading has been completed. please visit your Lab."
    send.login_to_email()
    send.send_email_with_timestamp(subject_email, body_email)
    print("\nScript Completed:")

if __name__ == '__main__':
    main()


....5%....10%....15%....20%....25%....30%....35%....40%....45%....50%....55%....60%....65%....70%....75%....80%....85%....90%....95%....100%Download Complete!
Report:
Total:3799, Success:2870, Failed:929
.Email has been sent.
you are logout

Script Completed:


In [13]:
idfilepath =  'binary_classes_ids.tsv'
with open(idfilepath, 'r') as idfile,  open('binary_classes_ids.csv', 'wt') as fcsv:
    # Dataset's name in which downloaded tweet-text will be saved
    tweetWriterfcsv = csv.writer(fcsv)
    tweetWriterfcsv.writerow(['tweetID', 'tweet-class'])
    for line in idfile:
#       print(line.split( ))
        tweetWriterfcsv.writerow(line.split( ))

In [20]:
import simplejson as json
tweets_downloaded = {1:'test'}
successTweetsFileJson = 'myTweet.json'
with open(successTweetsFileJson, 'w') as successTweetsFileJsonHandler:
  json.dump(tweets_downloaded, successTweetsFileJsonHandler)

In [ ]:
import pandas as pd
fname = 'tweet_failed_binaryCalsses.csv'
data = pd.DataFrame
data[data['Error-Message'].isin(["{""errors"":[{""code"":63,""message"":""User has been suspended.""}]}", 'Drama', 'Action'])].head(10)

In [49]:
import pandas as pd
fname = 'tweet_failed_binaryCalsses.csv'
data = pd.read_csv(fname)

rem = data['Error-Message'].drop_duplicates()
rem.to_csv('error.csv')

In [48]:
data['Error-Message'].describe()

count                                                   960
unique                                                    6
top       {"errors":[{"code":144,"message":"No status fo...
freq                                                    497
Name: Error-Message, dtype: object